In [ ]:
#在Colab用
!pip install -Uq timm==0.9.12


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.4 MB/s eta 0:00:00


In [ ]:
#授權
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
import torch.nn.functional as F
import timm

In [ ]:
num_epochs = 50
batch_size = 16
early_stop_patience = 4
patience_counter = 0
loss_alpha=1.0
loss_beta=0.5

In [ ]:
base_dir = "/content/drive/MyDrive/DLA_term_project_data/classified_data"

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
imagenet_stats = [(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)]
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_stats[0], imagenet_stats[1])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

validation_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_stats[0], imagenet_stats[1])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_stats[0], imagenet_stats[1])
])

train_dataset = datasets.ImageFolder(os.path.join(base_dir, 'train'), transform=train_transform)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, 'validation'), transform=validation_transform)
test_data = datasets.ImageFolder(os.path.join(base_dir, "test"), transform=test_transform)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print("類別數量：", len(train_dataset.classes))

類別數量： 22


In [ ]:
# customed loss function
# ----- 1. Node-to-Node Distance Matrix -----
node_distance_matrix = torch.tensor([
    [0,1,2,2,1,2,2,2.5,3.5,4,5.5,5,4,2.5,3.5,4,5,3.5,4.5,3.5,2.5,2.5],
    [1,0,1,3,2,3,1,1.5,2.5,3,4.5,4,3,1.5,4.5,5,6,4.5,5.5,4.5,3.5,3.5],
    [2,1,0,2,3,4,2,2.5,3.5,4,5.5,3,2,0.5,3.5,4,5,3.5,4.5,5.5,4.5,2.5],
    [2,3,2,0,1,2,4,4.5,5.5,6,7.5,5,4,2.5,1.5,2,3,1.5,2.5,3.5,2.5,0.5],
    [1,2,3,1,0,1,3,3.5,4.5,5,6.5,6,5,3.5,2.5,3,4,2.5,3.5,2.5,1.5,1.5],
    [2,3,4,2,1,0,2,2.5,3.5,4,5.5,7,6,4.5,3.5,4,5,3.5,2.5,1.5,0.5,2.5],
    [2,1,2,4,3,2,0,0.5,1.5,2,3.5,5,4,2.5,5.5,6,7,5.5,4.5,3.5,2.5,4.5],
    [2.5,1.5,2.5,4.5,3.5,2.5,0.5,0,1,1.5,3,3.5,4.5,3,6,6.5,7.5,6,5,4,3,5],
    [3.5,2.5,3.5,5.5,4.5,3.5,1.5,1,0,0.5,2,2.5,3.5,4,7,7.5,8.5,7,6,5,4,6],
    [4,3,4,6,5,4,2,1.5,0.5,0,1.5,2,3,4.5,7.5,8,9,7.5,6.5,5.5,4.5,6.5],
    [5.5,4.5,5.5,7.5,6.5,5.5,3.5,3,2,1.5,0,0.5,1.5,3,7,7.5,8.5,7,8,7,6,6.5],
    [5,4,3,5,6,7,5,3.5,2.5,2,0.5,0,1,2.5,6.5,7,8,6.5,7.5,7.5,6.5,5.5],
    [4,3,2,4,5,6,4,4.5,3.5,3,1.5,1,0,1.5,5.5,6,7,5.5,6.5,7.5,7,4.5],
    [2.5,1.5,0.5,2.5,3.5,4.5,2.5,3,4,4.5,3,2.5,1.5,0,4,4.5,5.5,4,5,6,5.5,3],
    [3.5,4.5,3.5,1.5,2.5,3.5,5.5,6,7,7.5,7,6.5,5.5,4,0,0.5,1.5,0.5,1.5,2.5,3.5,1],
    [4,5,4,2,3,4,6,6.5,7.5,8,7.5,7,6,4.5,0.5,0,1,1,2,3,4,1.5],
    [5,6,5,3,4,5,7,7.5,8.5,9,8.5,8,7,5.5,1.5,1,0,1.5,2.5,3.5,4.5,2.5],
    [3.5,4.5,3.5,1.5,2.5,3.5,5.5,6,7,7.5,7,6.5,5.5,4,0.5,1,1.5,0,1,2,3,1],
    [4.5,5.5,4.5,2.5,3.5,2.5,4.5,5,6,6.5,8,7.5,6.5,5,1.5,2,2.5,1,0,1,2,2],
    [3.5,4.5,5.5,3.5,2.5,1.5,3.5,4,5,5.5,7,7.5,7.5,6,2.5,3,3.5,2,1,0,1,3],
    [2.5,3.5,4.5,2.5,1.5,0.5,2.5,3,4,4.5,6,6.5,7,5.5,3.5,4,4.5,3,2,1,0,3],
    [2.5,3.5,2.5,0.5,1.5,2.5,4.5,5,6,6.5,6.5,5.5,4.5,3,1,1.5,2.5,1,2,3,3,0]
], dtype=torch.float)

# ----- 2. Area-to-Area Distance Matrix -----
area_distance_matrix = torch.tensor([
    [0, 1, 2, 2, 2, 2],
    [1, 0, 1, 1, 1, 1],
    [2, 1, 0, 1, 3, 3],
    [2, 1, 1, 0, 3, 3],
    [2, 1, 3, 3, 0, 1],
    [2, 1, 3, 3, 1, 0]
], dtype=torch.float)

# ----- 3. Node-to-Area Mapping -----
node_to_area = torch.tensor([
    0, 0, 0, 0, 0, 0, 0,    # 1~7 → area 0
    1,                      # 8  → area 1
    2, 2,                   # 9~10 → area 2
    3, 3, 3,                # 11~13 → area 3
    1,                      # 14 → area 1
    4, 4, 4,                # 15~17 → area 4
    5, 5, 5,                # 18~20 → area 5
    1,                      # 21 → area 1
    1                       # 22 → area 1
], dtype=torch.long)

# ----- 4. Custom Loss Function -----
class DistancePenaltyLoss(nn.Module):
    def __init__(self, node_distance_matrix, area_distance_matrix, node_to_area, alpha=1.0, beta=1.0):
        super().__init__()
        self.node_distance_matrix = node_distance_matrix
        self.area_distance_matrix = area_distance_matrix
        self.node_to_area = node_to_area
        self.alpha = alpha
        self.beta = beta
        self.ce = nn.CrossEntropyLoss()

    def forward(self, logits, targets):
        """
        logits: (batch_size, 22)
        targets: (batch_size,) - ground truth class indices
        """
        ce_loss = self.ce(logits, targets)

        probs = F.softmax(logits, dim=1)  # (B, 22)
        batch_size = targets.size(0)

        node_penalty = 0.0
        area_penalty = 0.0

        for i in range(batch_size):
            target = targets[i]
            prob = probs[i]  # shape (22,)

            # Node distance penalty
            node_distances = self.node_distance_matrix[target]  # (22,)
            node_penalty += (prob * node_distances).sum()

            # Area distance penalty
            target_area = self.node_to_area[target]
            pred_areas = self.node_to_area  # shape (22,)
            area_dists = self.area_distance_matrix[target_area][pred_areas]  # shape (22,)
            area_penalty += (prob * area_dists).sum()

        node_penalty /= batch_size
        area_penalty /= batch_size

        total_loss = ce_loss + self.alpha * node_penalty + self.beta * area_penalty
        return total_loss

In [ ]:
# timm.list_models('convnext*')
# timm.list_models('convnext*', pretrained=True)
# timm.list_models('efficient*', pretrained=True)
# timm.list_models('res*', pretrained=True)
# timm.list_models('swin*', pretrained=True)

# convnextv2_tiny.fcmae
# efficientvit_b0.r224_in1k
# efficientvit_b1.r224_in1k
# efficientvit_b2.r224_in1k
# efficientvit_b3.r224_in1k
# efficientvit_m0.r224_in1k
# efficientvit_m1.r224_in1k
# efficientvit_m2.r224_in1k
# efficientvit_m3.r224_in1k
# efficientvit_m4.r224_in1k
# efficientvit_m5.r224_in1k
# efficientformerv2_s0.snap_dist_in1k
# efficientformerv2_s1.snap_dist_in1k
# efficientformerv2_s2.snap_dist_in1k
# resnet50.a1_in1k
# resnet50d.ra2_in1k
# resnet50.fb_swsl_ig1b_ft_in1k
# resnet50.tv_in1k
# resnet50.ra_in1k
# res2net50d.in1k
# res2net50_14w_8s.in1k
# mobilevitv2_100.cvnets_in1k
# mobilevitv2_150.cvnets_in22k_ft_in1k
# mobilenetv3_large_100.ra_in1k
# mobileone_s1.apple_in1k
# mobilenetv2_100.ra_in1k
# swin_tiny_patch4_window7_224.ms_in1k
# swinv2_tiny_window8_256.ms_in1k
# swinv2_cr_tiny_ns_224.sw_in1k



# ✅ Overall Best Choices (Balanced: Accuracy, Speed, Size)
# mobilevitv2_100.cvnets_in1k	       Transformer-CNN hybrid, small but very accurate
# efficientnet_b0	             Lightweight, easy to train, great accuracy/size tradeoff
# mobilenetv3_large_100.ra_in1k       Very fast, widely supported, good baseline for small datasets
# res2net50d.in1k	             Strong feature extractor, better than plain ResNet50
# resnet50d.ra2_in1k             Modern ResNet variant with strong augmentations
# swinv2_tiny_window8_256.ms_in1k	     Lightweight transformer, good if you want to try Swin family

# 🧠 If You Want Transfer Learning Strength
# mobilevitv2_150.cvnets_in22k_ft_in1k	  Pretrained on ImageNet-22K → fine-tuned on IN1K
# resnet50.fb_swsl_ig1b_ft_in1k	       Trained on IG-1B dataset — great features even for few-shot

# 🚀 If You Want Ultra-Fast Training
# mobileone_s1.apple_in1k          	Tiny, deployable, optimized for fast inference
# mobilenetv2_100.ra_in1k         	Classic, efficient, and still performs well

In [ ]:
# ---------- 100% MLP baseline：Patch-MLP ----------

#希望是好的baseline哈哈 或是你們如果有其他想法也可以試試看ㄛ
#大概的想法：把影像切成 (patch_size x patch_size) 小塊 → 線性投影 → 全域平均 → MLP 分類



import torch
import torch.nn as nn
import torch.nn.functional as F

class PureMLP(nn.Module):

    def __init__(self, num_classes=22, patch_size=16, proj_dim=256,
                 mlp_hidden=[512, 256], dropout=0.2):
        super().__init__()
        self.patch_size = patch_size
        patch_dim = 3 * patch_size * patch_size          # 每個 patch 的展開向量長度
        self.unfold = nn.Unfold(kernel_size=patch_size,
                                stride=patch_size)       # 切 patch

        # ① patch projector（把每個 patch 投影到 proj_dim 維度）
        self.proj = nn.Sequential(
            nn.Linear(patch_dim, proj_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )

        # ② 圖像層級全域特徵 = 所有 patch 向量的平均
        # ③ MLP 分類頭
        mlp_layers = []
        in_dim = proj_dim
        for h in mlp_hidden:
            mlp_layers += [nn.Linear(in_dim, h), nn.GELU(), nn.Dropout(dropout)]
            in_dim = h
        mlp_layers.append(nn.Linear(in_dim, num_classes))
        self.mlp_head = nn.Sequential(*mlp_layers)

    def forward(self, x):
        # x: (B, 3, H, W)  →  unfold → (B, patch_dim, N_patches)
        patches = self.unfold(x)                         # 切 patch
        patches = patches.transpose(1, 2)                # (B, N, patch_dim)
        patches = self.proj(patches)                     # (B, N, proj_dim)
        feat = patches.mean(dim=1)                       # (B, proj_dim)  全域平均
        logits = self.mlp_head(feat)                     # (B, num_classes)
        return logits


In [ ]:
# Step 5: 初始化模型與優化器
#模型換成 PureMLP baseline
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = PureMLP(
    num_classes=len(train_dataset.classes),
    patch_size=16,       # 與 PureMLP 內預設一致
    proj_dim=256,
    mlp_hidden=[512, 256],
    dropout=0.2
).to(device)

criterion = DistancePenaltyLoss(
    node_distance_matrix.to(device),
    area_distance_matrix.to(device),
    node_to_area.to(device),
    alpha=loss_alpha,
    beta=loss_beta
)

optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scaler = GradScaler()


<ipython-input-9-c8ba2027748c>:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
def train_one_epoch_amp(model, loader):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        with torch.autocast(device.type if device.type != "mps" else "cpu", enabled=(device.type != "cpu")):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        # Print batch details
        print(f"Batch {batch_idx+1}/{len(loader)} | Loss: {loss.item():.4f} | Batch Acc: {(predicted == labels).float().mean().item():.4f}")
    acc = correct / total
    print(f"Epoch Train Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f"Validation Accuracy: {acc:.4f}")
    return acc

In [ ]:
# Step 7: 訓練流程
best_acc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_one_epoch_amp(model, train_loader)
    val_acc = evaluate(model, val_loader)
    # print("validation accuracy: ", val_acc)
    if val_acc > best_acc:
        best_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
        print(f"✅ 儲存最佳模型（Acc: {best_acc:.4f}）")
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print("⏹️ 觸發 Early Stopping")
            break

test_acc = evaluate(model, test_loader)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")


Epoch 1/50
Batch 1/72 | Loss: 7.3525 | Batch Acc: 0.0625
Batch 2/72 | Loss: 7.4870 | Batch Acc: 0.1250
Batch 3/72 | Loss: 7.4404 | Batch Acc: 0.0000
Batch 4/72 | Loss: 7.3996 | Batch Acc: 0.0000
Batch 5/72 | Loss: 7.1873 | Batch Acc: 0.1250
Batch 6/72 | Loss: 7.6169 | Batch Acc: 0.0000
Batch 7/72 | Loss: 7.3677 | Batch Acc: 0.1250
Batch 8/72 | Loss: 7.0879 | Batch Acc: 0.0625
Batch 9/72 | Loss: 7.4478 | Batch Acc: 0.0625
Batch 10/72 | Loss: 7.1644 | Batch Acc: 0.1250
Batch 11/72 | Loss: 7.3401 | Batch Acc: 0.0000
Batch 12/72 | Loss: 7.3415 | Batch Acc: 0.0000
Batch 13/72 | Loss: 7.5210 | Batch Acc: 0.0625
Batch 14/72 | Loss: 7.5104 | Batch Acc: 0.0625
Batch 15/72 | Loss: 7.6776 | Batch Acc: 0.0000
Batch 16/72 | Loss: 7.4344 | Batch Acc: 0.0625
Batch 17/72 | Loss: 7.2264 | Batch Acc: 0.0625
Batch 18/72 | Loss: 7.3690 | Batch Acc: 0.0625
Batch 19/72 | Loss: 6.9677 | Batch Acc: 0.0000
Batch 20/72 | Loss: 7.6319 | Batch Acc: 0.0625
Batch 21/72 | Loss: 7.1979 | Batch Acc: 0.0625
Batch 22/7

In [ ]:
test_acc = evaluate(model, test_loader)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

Validation Accuracy: 0.1136

Final Test Accuracy: 0.1136


In [ ]:
# Step 8: 載入最佳模型（推論前）
model.load_state_dict(torch.load("best_model.pth"))
model.to(device)

PureMLP(
  (unfold): Unfold(kernel_size=16, dilation=1, padding=0, stride=16)
  (proj): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.2, inplace=False)
  )
  (mlp_head): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=256, out_features=22, bias=True)
  )
)

In [ ]:
from PIL import Image

# Load model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
model.to(device)

# Load and preprocess image
img_path = "/content/drive/MyDrive/DLA_term_project_data/classified_data/test/node_15/IMG_4393.jpg"
img = Image.open(img_path).convert("RGB")
input_tensor = test_transform(img).unsqueeze(0).to(device)

# Predict
with torch.no_grad():
    logits = model(input_tensor)
    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

# Print probabilities
for idx, prob in enumerate(probs):
    print(f"Node {idx+1}: {prob:.4f}")

Node 1: 0.0342
Node 2: 0.0413
Node 3: 0.0544
Node 4: 0.0614
Node 5: 0.0537
Node 6: 0.0879
Node 7: 0.0553
Node 8: 0.0328
Node 9: 0.0171
Node 10: 0.0212
Node 11: 0.0150
Node 12: 0.0171
Node 13: 0.0184
Node 14: 0.0511
Node 15: 0.0407
Node 16: 0.0243
Node 17: 0.0181
Node 18: 0.0331
Node 19: 0.0697
Node 20: 0.0582
Node 21: 0.0644
Node 22: 0.1306


In [ ]:
import numpy as np
from collections import Counter

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print predictions and ground truths
for i, (pred, label) in enumerate(zip(all_preds, all_labels)):
    correct = "✅" if pred == label else "❌"
    print(f"Sample {i+1}: True Node={label+1}, Predicted Node={pred+1} {correct}")

# Count misclassifications per node
misclass_counts = Counter()
for pred, label in zip(all_preds, all_labels):
    if pred != label:
        misclass_counts[label+1] += 1  # +1 for 1-based node index

print("\nMisclassification count per node (True label):")
for node in range(1, len(test_loader.dataset.classes)+1):
    print(f"Node {node}: {misclass_counts[node]}")

# Find the most misclassified node(s)
if misclass_counts:
    most_misclassified = misclass_counts.most_common(1)[0]
    print(f"\nNode most often misclassified: Node {most_misclassified[0]} ({most_misclassified[1]} times)")
else:
    print("\nNo misclassifications!")

Sample 1: True Node=1, Predicted Node=22 ❌
Sample 2: True Node=1, Predicted Node=2 ❌
Sample 3: True Node=1, Predicted Node=21 ❌
Sample 4: True Node=1, Predicted Node=19 ❌
Sample 5: True Node=1, Predicted Node=2 ❌
Sample 6: True Node=1, Predicted Node=22 ❌
Sample 7: True Node=1, Predicted Node=22 ❌
Sample 8: True Node=1, Predicted Node=2 ❌
Sample 9: True Node=2, Predicted Node=22 ❌
Sample 10: True Node=2, Predicted Node=22 ❌
Sample 11: True Node=2, Predicted Node=22 ❌
Sample 12: True Node=2, Predicted Node=22 ❌
Sample 13: True Node=2, Predicted Node=2 ✅
Sample 14: True Node=2, Predicted Node=2 ✅
Sample 15: True Node=2, Predicted Node=22 ❌
Sample 16: True Node=2, Predicted Node=9 ❌
Sample 17: True Node=3, Predicted Node=22 ❌
Sample 18: True Node=3, Predicted Node=19 ❌
Sample 19: True Node=3, Predicted Node=19 ❌
Sample 20: True Node=3, Predicted Node=21 ❌
Sample 21: True Node=3, Predicted Node=2 ❌
Sample 22: True Node=3, Predicted Node=22 ❌
Sample 23: True Node=3, Predicted Node=22 ❌
Samp